In [ ]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

In [ ]:
import pickle
from fitter import Fitter
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats

In [ ]:
#CREATE DISTRIBUTIONS ARRAY

pdfs = pickle.load(open("./distributions/pdfs.pickle", "rb"))
data = pd.read_csv('./full_data_with_nans.csv')
col_names = np.array(data.keys())
#ST STARTS AT 49
st_cols = col_names[49:-2]

bad_indices = []

#THESE TWO ARE BAD IN GENERAL
print(np.argwhere(st_cols=='ST8SV')[0][0])
print(np.argwhere(st_cols=='ST68SV')[0][0])

#THESE THREE ARE MIXTURE
bad_indices.append(np.argwhere(st_cols=='ST21SV')[0][0])
bad_indices.append(np.argwhere(st_cols=='ST80SV')[0][0])
bad_indices.append(np.argwhere(st_cols=='ST153SV')[0][0])

distributions = []

mixture_params = []
mixture_params.append( (-0.0033, 0.0006, 1.2/11.2, 0.0004, 0.0013, 10/11.2) )
mixture_params.append( (-0.0026, 0.0004, 1.2/11.2, 0.00037, 0.001, 10/11.2) )
mixture_params.append( (-5.352063451854405e-05, 2.9170065357038982e-06, 1.3/9.3, 8.993002770407149e-06, 8.263642179231052e-06, 8/9.3) )

for pdf in pdfs:
    distributions.append(pdf[1].get_best())

for j, i in enumerate(bad_indices):
    key = list(distributions[i].keys())[0]
    distributions[i]['normal_mixture'] = distributions[i].pop(key)
    distributions[i]['normal_mixture'] = mixture_params[j]
    print(distributions[i])

for i, pdf in enumerate(pdfs):
    distributions[i]['st'] = st_cols[i]
    
print(len(distributions))

In [ ]:
#CREATE LATENT SPACE
#THIS ALSO PLOTS HISTOGRAMS AND DISTRIBUTIONS FROM LESHA'S PICKLE
# 325 histograms

import scipy.stats
import numpy as np
import numpy.random
import scipy.stats as ss
import matplotlib.pyplot as plt

data = pd.read_csv('./st_features_without_nans_and_y_with_nans.csv')
latent_features = np.zeros((data.shape[0], data.shape[1]-1))
latent_unified = np.zeros((data.shape[0], data.shape[1]-1))
col_names = np.array(data.keys())
st_cols = col_names[1:]

for i in range(len(st_cols)):
    current = data[st_cols[i]]
    
    mean = np.mean(current)
    stdev = np.std(current)
    current = (current - mean)/stdev**2
    
    plt.figure()
    plt.hist(current, bins = 100, density=True)

    key = list(distributions[i].keys())[0]
    if key == 'normal_mixture':
        n = 10000
        numpy.random.seed(0x5eed)
        norm_params = np.array([[distributions[i][key][0],
                                 distributions[i][key][1]],
                                [distributions[i][key][3],
                                 distributions[i][key][4]]])
        weights = np.array([distributions[i][key][2],
                            distributions[i][key][5]])
        n_components = norm_params.shape[0]
        mixture_idx = numpy.random.choice(len(weights), size=n, replace=True, p=weights)
        y = numpy.fromiter((ss.norm.rvs(*(norm_params[i])) for i in mixture_idx),
                           dtype=np.float64)
        xs = np.linspace(y.min(), y.max(), 200)
        ys = np.zeros_like(xs)

        for (l, s), w in zip(norm_params, weights):
            ys += ss.norm.pdf(xs, loc=l, scale=s) * w
        plt.plot(xs, ys, lw=4)
        for j in range(latent_features.shape[0]):
            val = 0.0
            for (l, s), w in zip(norm_params, weights):
                val += ss.norm.cdf(current[j], loc=l, scale=s) * w
            latent_features[j][i] = val
            latent_unified[j][i] = scipy.stats.norm.ppf(latent_features[j][i], loc=0., scale=1.)
    else:
        dist = getattr(scipy.stats, key)(*distributions[i][key])
        y = np.linspace(dist.ppf(0.01), dist.ppf(0.99))
        plt.plot(y, dist.pdf(y), lw=4)
        for j in range(latent_features.shape[0]):
            latent_features[j][i] = dist.cdf(current[j])
            latent_unified[j][i] = scipy.stats.norm.ppf(latent_features[j][i], loc=0., scale=1.)

    plt.title(st_cols[i])
    plt.savefig("./plots/" + st_cols[i] + ".png")
    plt.close()
    
for i in range(len(st_cols)):
    data[st_cols[i]] = latent_unified.T[i]
    #print(data[st_cols[i]])
print(data.shape)
data.to_csv("latent_unified_with_nan_y_with_nan.csv", index=False)


In [ ]:
#CHECK IF DATA IS NOW STANDARD NORMAL
#PLOT 325 HISTOGRAMS AND STANDARD NORMAL DISTRIBUTION

#ST91CV ST91TA ARE BAD, THEY ADD INFS
data_new = data.replace([np.inf, -np.inf], np.nan)
data_new = data_new.dropna(subset=col_names)

norm = scipy.stats.norm(loc = 0., scale=1.)
z = np.linspace(norm.ppf(0.01), norm.ppf(0.99), col_names.shape[0])
zz = norm.pdf(z)

for i in range(1, col_names.shape[0]):
    cur = data_new[col_names[i]]
    plt.figure(figsize=(20, 10))
    plt.hist(cur, bins=200, density = True)
    plt.plot(z, zz, lw=4)
    plt.title(col_names[i])
    plt.savefig("./latent/"+col_names[i]+".png")
    
    plt.close()